<a href="https://colab.research.google.com/github/alfaizzz786-design/Land-Registry-Property-Ownership-Ledger/blob/main/Land_Registry_Property_Ownership_Ledger.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json, time, os
from dataclasses import dataclass, asdict
from hashlib import sha256
from typing import List, Optional

DATA_FILE = "land_chain.json"

@dataclass
class Deed:
    deed_id: str
    owner: str
    property_address: str
    registered_on: str
    registrar: str
    status: str
    remarks: str = ""

    def serialize(self) -> str:
        payload = {
            "deed_id": self.deed_id,
            "owner": self.owner,
            "property_address": self.property_address,
            "registered_on": self.registered_on,
            "registrar": self.registrar,
            "status": self.status,
            "remarks": self.remarks,
        }
        return json.dumps(payload, separators=(",", ":"), sort_keys=True)

@dataclass
class Block:
    index: int
    timestamp: float
    data: dict
    prev_hash: str
    nonce: int = 0

    def hash(self) -> str:
        body = {
            "index": self.index,
            "timestamp": self.timestamp,
            "data": self.data,
            "prev_hash": self.prev_hash,
            "nonce": self.nonce,
        }
        return sha256(json.dumps(body, separators=(",", ":"), sort_keys=True).encode()).hexdigest()

class Blockchain:
    def __init__(self):
        self.chain: List[Block] = []
        self.load() or self._create_genesis()

    def _create_genesis(self):
        genesis = Block(
            index=0,
            timestamp=time.time(),
            data={"type": "GENESIS", "msg": "Land Registry Ledger Genesis"},
            prev_hash="0"*64,
        )
        self.chain = [genesis]
        self.save()

    def last_block(self) -> Block:
        return self.chain[-1]

    def add_block(self, data: dict) -> Block:
        new_block = Block(
            index=len(self.chain),
            timestamp=time.time(),
            data=data,
            prev_hash=self.last_block().hash()
        )
        while not new_block.hash().startswith("00"):  # proof-of-work
            new_block.nonce += 1
        self.chain.append(new_block)
        self.save()
        return new_block

    def is_valid(self) -> bool:
        if not self.chain:
            return False
        for i in range(1, len(self.chain)):
            cur, prev = self.chain[i], self.chain[i-1]
            if cur.prev_hash != prev.hash():
                return False
            if not cur.hash().startswith("00"):
                return False
        return True

    def save(self):
        with open(DATA_FILE, "w") as f:
            out = [asdict(b) for b in self.chain]
            json.dump(out, f, indent=2)

    def load(self) -> bool:
        if not os.path.exists(DATA_FILE):
            return False
        with open(DATA_FILE, "r") as f:
            raw = json.load(f)
            self.chain = [Block(**b) for b in raw]
        return True

    def find_latest_deed(self, deed_id: str) -> Optional[Deed]:
        for b in reversed(self.chain):
            if b.data.get("type") in ("ISSUE", "TRANSFER", "REVOKE"):
                d = b.data["deed"]
                if d["deed_id"] == deed_id:
                    return Deed(**d)
        return None

    def issue_deed(self, d: Deed) -> str:
        existing = self.find_latest_deed(d.deed_id)
        if existing and existing.status == "ISSUED":
            return f"[X] Deed {d.deed_id} already exists."
        data = {"type": "ISSUE", "deed": json.loads(d.serialize())}
        blk = self.add_block(data)
        return f"[✓] Registered deed {d.deed_id} in block #{blk.index}."

    def transfer_deed(self, deed_id: str, new_owner: str, registrar: str) -> str:
        current = self.find_latest_deed(deed_id)
        if not current:
            return f"[X] Deed {deed_id} not found."
        if current.status == "REVOKED":
            return f"[X] Deed {deed_id} is revoked and cannot be transferred."
        transferred = Deed(
            deed_id=deed_id,
            owner=new_owner,
            property_address=current.property_address,
            registered_on=current.registered_on,
            registrar=registrar,
            status="ISSUED",
            remarks=f"Transferred from {current.owner} to {new_owner}"
        )
        data = {"type": "TRANSFER", "deed": json.loads(transferred.serialize())}
        blk = self.add_block(data)
        return f"[✓] Ownership of {deed_id} transferred to {new_owner} in block #{blk.index}."

    def revoke_deed(self, deed_id: str, registrar: str, remarks: str) -> str:
        current = self.find_latest_deed(deed_id)
        if not current:
            return f"[X] Deed {deed_id} not found."
        if current.status == "REVOKED":
            return f"[X] Deed {deed_id} already REVOKED."
        revoked = Deed(
            deed_id=deed_id,
            owner=current.owner,
            property_address=current.property_address,
            registered_on=current.registered_on,
            registrar=registrar,
            status="REVOKED",
            remarks=remarks
        )
        data = {"type": "REVOKE", "deed": json.loads(revoked.serialize())}
        blk = self.add_block(data)
        return f"[✓] Revoked deed {deed_id} in block #{blk.index}."

    def verify_deed(self, deed_id: str) -> str:
        d = self.find_latest_deed(deed_id)
        if not d:
            return f"[?] {deed_id} not found in ledger."
        for b in reversed(self.chain):
            if b.data.get("type") in ("ISSUE", "TRANSFER", "REVOKE"):
                dd = b.data["deed"]
                if dd["deed_id"] == deed_id:
                    ok_link = b.prev_hash == self.chain[b.index-1].hash() if b.index > 0 else True
                    ok_pow = b.hash().startswith("00")
                    state = f"VALID LINK={ok_link}, POW={ok_pow}"
                    return f"[INFO] {deed_id}: status={dd['status']} (owner={dd['owner']}). {state}."
        return f"[?] {deed_id} not found in ledger."

def menu():
    bc = Blockchain()
    while True:
        print("\n--- Land Registry Ledger ---")
        print("1) Register new deed")
        print("2) Transfer ownership")
        print("3) Revoke deed")
        print("4) Verify deed")
        print("5) Show chain length & validity")
        print("6) List last N blocks")
        print("0) Exit")
        choice = input("Select: ").strip()

        if choice == "1":
            did = input("Deed ID: ").strip()
            owner = input("Owner: ").strip()
            addr = input("Property Address: ").strip()
            date = input("Registered on (YYYY-MM-DD): ").strip()
            registrar = input("Registrar: ").strip()
            msg = bc.issue_deed(Deed(did, owner, addr, date, registrar, "ISSUED"))
            print(msg)

        elif choice == "2":
            did = input("Deed ID to transfer: ").strip()
            new_owner = input("New owner: ").strip()
            registrar = input("Registrar confirming: ").strip()
            print(bc.transfer_deed(did, new_owner, registrar))

        elif choice == "3":
            did = input("Deed ID to revoke: ").strip()
            registrar = input("Registrar revoking: ").strip()
            remarks = input("Reason/remarks: ").strip()
            print(bc.revoke_deed(did, registrar, remarks))

        elif choice == "4":
            did = input("Deed ID to verify: ").strip()
            print(bc.verify_deed(did))

        elif choice == "5":
            print(f"Blocks: {len(bc.chain)} | Valid: {bc.is_valid()}")

        elif choice == "6":
            try:
                n = int(input("How many recent blocks? ").strip())
            except ValueError:
                n = 5
            for b in bc.chain[-n:]:
                print(f"#{b.index} ts={int(b.timestamp)} prev[:8]={b.prev_hash[:8]} nonce={b.nonce}")
                print(f"   type={b.data.get('type')}")
                if b.data.get("type") in ("ISSUE", "TRANSFER", "REVOKE"):
                    d = b.data["deed"]
                    print(f"   id={d['deed_id']} status={d['status']} owner={d['owner']}")

        elif choice == "0":
            break
        else:
            print("Invalid choice.")

if __name__ == "__main__":
    menu()



--- Land Registry Ledger ---
1) Register new deed
2) Transfer ownership
3) Revoke deed
4) Verify deed
5) Show chain length & validity
6) List last N blocks
0) Exit
Select: 1
Deed ID:  DEED-1001
Owner: John Smith
Property Address:  45 Lakeview Road
Registered on (YYYY-MM-DD): 2025-03-01
Registrar:  City Registrar Office
[✓] Registered deed DEED-1001 in block #1.

--- Land Registry Ledger ---
1) Register new deed
2) Transfer ownership
3) Revoke deed
4) Verify deed
5) Show chain length & validity
6) List last N blocks
0) Exit
Select: 4
Deed ID to verify: DEED-1001
[INFO] DEED-1001: status=ISSUED (owner=John Smith). VALID LINK=True, POW=True.

--- Land Registry Ledger ---
1) Register new deed
2) Transfer ownership
3) Revoke deed
4) Verify deed
5) Show chain length & validity
6) List last N blocks
0) Exit
Select: 2
Deed ID to transfer:  DEED-1001
New owner:  Alice Johnson
Registrar confirming:  City Registrar Office
[✓] Ownership of DEED-1001 transferred to Alice Johnson in block #2.

--- 